AUTHORS: Ludwig Wideskär (luai18@student.bth.se), Akshaya Bathula (akba21@student.bth.se)

---
Import libraries:

In [43]:
!python --version

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import array

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from datetime import *
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score
from tabulate import tabulate

Python 3.8.10


---
Import the dataset and setting relevant column names:

In [44]:
Dataset = pd.read_csv("/content/spambase.data")
# Read the ".data" file as if it was a csv file but keep the .data file ending
# Make sure the database can be read before executing.

Dataset.columns = ['word_freq_make', 'word_freq_address', 'word_freq_all',
                   'word_freq_3d', 'word_freq_our', 'word_freq_over',
                   'word_freq_remove', 'word_freq_internet', 'word_freq_order',
                   'word_freq_mail', 'word_freq_receive', 'word_freq_will',
                   'word_freq_people', 'word_freq_report',
                   'word_freq_addresses', 'word_freq_free',
                   'word_freq_business', 'word_freq_email', 'word_freq_you',
                   'word_freq_credit', 'word_freq_your', 'word_freq_font',
                   'word_freq_000', 'word_freq_money', 'word_freq_hp',
                   'word_freq_hpl', 'word_freq_george', 'word_freq_650',
                   'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
                   'word_freq_857', 'word_freq_data', 'word_freq_415',
                   'word_freq_85', 'word_freq_technology', 'word_freq_1999',
                   'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
                   'word_freq_cs', 'word_freq_meeting', 'word_freq_original',
                   'word_freq_project', 'word_freq_re', 'word_freq_edu',
                   'word_freq_table', 'word_freq_conference', 'word_freq_;',
                   'word_freq_(', 'word_freq_[', 'word_freq_!', 'word_freq_$',
                   'word_freq_#', 'capital_run_length_average',
                   'capital_run_length_longest', 'capital_run_length_total',
                   'is_spam']


---
The positive literals which are spam in 
the data are taken as pos_res. The variable x denotes that the last 3 columns in the dataset are ignored as it contains expectionally high values compared to the number of bins.

In [45]:
x = Dataset.iloc[:,: -3]
y = Dataset.iloc[:,-1]
pos_res = x.iloc[: 1812] 
neg_res = x.iloc[1813 :]

In [46]:
kfolds = KFold(n_splits=10, shuffle=True)

list_kfold_time1 = []
list_kfold_time2 = []
list_kfold_time3 = []

list_kfold_acc1 = []
list_kfold_acc2 = []
list_kfold_acc3 = []

list_kfold_fm1 = []
list_kfold_fm2 = []
list_kfold_fm3 = []

for train_index, test_index in kfolds.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    # K-Nearest Neighbors (K-NN)
    knn = KNeighborsClassifier()
    strt_time1 = datetime.now()
    knn.fit(x_train, y_train)
    end_time1 = datetime.now()
    time1 = end_time1 - strt_time1
    tym1 = time1.total_seconds()
    list_kfold_time1.append(tym1)

    y_pred = knn.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    list_kfold_acc1.append(accuracy)

    f_measure = f1_score(y_test, y_pred, average='weighted')
    list_kfold_fm1.append(f_measure)


    # Support Vector Machine (SVM)
    svc = SVC()
    strt_time2 = datetime.now()
    svc.fit(x_train, y_train)
    end_time2 = datetime.now()
    time2 = end_time2 - strt_time2
    tym2 = time2.total_seconds()
    list_kfold_time2.append(tym2)

    y_pred = svc.predict(x_test)
    accuracy2 = accuracy_score(y_test, y_pred)
    list_kfold_acc2.append(accuracy2)

    f_measure2 = f1_score(y_test, y_pred, average='weighted')
    list_kfold_fm2.append(f_measure2)
    

    # Naive Bayes
    gnb = GaussianNB()
    strt_time3 = datetime.now()
    gnb.fit(x_train, y_train)
    end_time3 = datetime.now()
    time3 = end_time3 - strt_time3
    tym3 = time3.total_seconds()
    list_kfold_time3.append(tym3)

    y_pred = gnb.predict(x_test)
    accuracy3 = accuracy_score(y_test, y_pred)
    list_kfold_acc3.append(accuracy3)

    f_measure3 = f1_score(y_test, y_pred, average='weighted')
    list_kfold_fm3.append(f_measure3)

---
**Cross-validation function**

Performs cross-validation on the three input lists.

In [47]:
def cross_validation(list_alg1, list_alg2, list_alg3):
  headers = ['Fold', 'K-NN', 'SVM', 'Naive Bayes']

  dict_data={headers[1]:list_alg1, headers[2]:list_alg2, headers[3]:list_alg3}
  data_df=pd.DataFrame(dict_data)

  data_df.index = np.arange(1, len(data_df) + 1)

  data_df[headers[0]]=data_df.index
  col_fold = data_df.pop(headers[0])
  data_df.insert(0, headers[0], col_fold)

  avg_alg1 = sum(list_alg1)/len(list_alg1)
  avg_alg2 = sum(list_alg2)/len(list_alg2)
  avg_alg3 = sum(list_alg3)/len(list_alg3)

  stdev_alg1 = data_df[headers[1]].std()
  stdev_alg2 = data_df[headers[2]].std()
  stdev_alg3 = data_df[headers[3]].std()

  avg_and_stdev_df = pd.DataFrame([
      ['-', '-', '-', '-'],
      ['avg', avg_alg1, avg_alg2, avg_alg3],
      ['stdev', stdev_alg1, stdev_alg2, stdev_alg3]],
      columns=headers)

  result_df = pd.concat([data_df, avg_and_stdev_df])

  print(tabulate(result_df, headers="keys", showindex=False, tablefmt="rst"))


---
**Nemenyi test function**

Calculates the critical difference between average ranks for the algorithms

In [48]:
def Nemenyi_test(k, n, dict_avg_ranks):
  print("\nNemenyi test:")
  
  degrees_of_freedom = 3.314
  q_alpha = degrees_of_freedom / np.sqrt(2)
  critical_difference = q_alpha * np.sqrt((k*(k+1))/(6*n))
  print("Critical difference:", critical_difference)

  name_algs = []
  avg_ranks_algs = []

  for k in sorted(dict_avg_ranks, key = dict_avg_ranks.get):
    name_algs.append(k)
    avg_ranks_algs.append(dict_avg_ranks[k])

  diff1 = avg_ranks_algs[1] - avg_ranks_algs[0]
  diff2 = avg_ranks_algs[2] - avg_ranks_algs[0]
  diff3 = avg_ranks_algs[2] - avg_ranks_algs[1]

  if diff1 > critical_difference:
    if diff2 > critical_difference:
      print("The algorithm", name_algs[0], "performs significantly better than",
            name_algs[1], "and", name_algs[2],
            "with performance differences of:", diff1, "and", diff2,
            "respectively.")
    else:  
      print("The algorithm", name_algs[0], "performs significantly better than",
          name_algs[1], "with a performance difference of", diff1)
  else:    
    if diff2 > critical_difference:
      print("The algorithm", name_algs[0], "performs significantly better than",
            name_algs[2], "with a performance difference of", diff2)

  if diff3 > critical_difference:
    print("The algorithm", name_algs[1], "performs significantly better than",
          name_algs[2], "with a performance difference of", diff3)

---
**Friedman test function**

Performs Friedman test on the input lists

In [49]:
def Friedman_test(list_alg1, list_alg2, list_alg3, highest):
  if (len(list_alg1) == len(list_alg2) and len(list_alg1) == len(list_alg3)):
    list_size = len(list_alg1)

    list_rank_alg1 = []
    list_rank_alg2 = []
    list_rank_alg3 = []
    avg_rank_alg1 = 0
    avg_rank_alg2 = 0
    avg_rank_alg3 = 0

    if highest == True: # Highest value gains highest score
      for i in range(list_size):
        # Unique scenarios, duplicates removed

        if list_alg1[i] > list_alg2[i] and list_alg2[i] > list_alg3[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(2)
           list_rank_alg3.append(3)

        elif list_alg1[i] == list_alg2[i] and list_alg2[i] > list_alg3[i]:
           list_rank_alg1.append(1.5)
           list_rank_alg2.append(1.5)
           list_rank_alg3.append(3)

        elif list_alg1[i] > list_alg2[i] and list_alg2[i] == list_alg3[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(2.5)
           list_rank_alg3.append(2.5)

        elif list_alg1[i] == list_alg2[i] and list_alg2[i] == list_alg3[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(2)
           list_rank_alg3.append(2)

        # ----------

        elif list_alg1[i] > list_alg3[i] and list_alg3[i] > list_alg2[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(3)
           list_rank_alg3.append(2)

        elif list_alg1[i] == list_alg3[i] and list_alg3[i] > list_alg2[i]:
           list_rank_alg1.append(1.5)
           list_rank_alg2.append(3)
           list_rank_alg3.append(1.5)

        # ----------

        elif list_alg2[i] > list_alg1[i] and list_alg1[i] > list_alg3[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(1)
           list_rank_alg3.append(3)

        elif list_alg2[i] > list_alg1[i] and list_alg1[i] == list_alg3[i]:
           list_rank_alg1.append(2.5)
           list_rank_alg2.append(1)
           list_rank_alg3.append(2.5)

        # ----------

        elif list_alg2[i] > list_alg3[i] and list_alg3[i] > list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(1)
           list_rank_alg3.append(2)
           
        elif list_alg2[i] == list_alg3[i] and list_alg3[i] > list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(1.5)
           list_rank_alg3.append(1.5)
           
        # ----------

        elif list_alg3[i] > list_alg1[i] and list_alg1[i] > list_alg2[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(3)
           list_rank_alg3.append(1)
           
        elif list_alg3[i] > list_alg1[i] and list_alg1[i] == list_alg2[i]:
           list_rank_alg1.append(2.5)
           list_rank_alg2.append(2.5)
           list_rank_alg3.append(1)
           
        # ----------

        elif list_alg3[i] > list_alg2[i] and list_alg2[i] > list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(2)
           list_rank_alg3.append(1)

    elif highest == False: # Lowest value gains highest score
      for i in range(list_size):
        # Unique scenarios, duplicates removed

        if list_alg1[i] < list_alg2[i] and list_alg2[i] < list_alg3[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(2)
           list_rank_alg3.append(3)

        elif list_alg1[i] == list_alg2[i] and list_alg2[i] < list_alg3[i]:
           list_rank_alg1.append(1.5)
           list_rank_alg2.append(1.5)
           list_rank_alg3.append(3)

        elif list_alg1[i] < list_alg2[i] and list_alg2[i] == list_alg3[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(2.5)
           list_rank_alg3.append(2.5)

        elif list_alg1[i] == list_alg2[i] and list_alg2[i] == list_alg3[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(2)
           list_rank_alg3.append(2)

        # ----------

        elif list_alg1[i] < list_alg3[i] and list_alg3[i] < list_alg2[i]:
           list_rank_alg1.append(1)
           list_rank_alg2.append(3)
           list_rank_alg3.append(2)

        elif list_alg1[i] == list_alg3[i] and list_alg3[i] < list_alg2[i]:
           list_rank_alg1.append(1.5)
           list_rank_alg2.append(3)
           list_rank_alg3.append(1.5)

        # ----------

        elif list_alg2[i] < list_alg1[i] and list_alg1[i] < list_alg3[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(1)
           list_rank_alg3.append(3)

        elif list_alg2[i] < list_alg1[i] and list_alg1[i] == list_alg3[i]:
           list_rank_alg1.append(2.5)
           list_rank_alg2.append(1)
           list_rank_alg3.append(2.5)

        # ----------

        elif list_alg2[i] < list_alg3[i] and list_alg3[i] < list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(1)
           list_rank_alg3.append(2)
           
        elif list_alg2[i] == list_alg3[i] and list_alg3[i] < list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(1.5)
           list_rank_alg3.append(1.5)
           
        # ----------

        elif list_alg3[i] < list_alg1[i] and list_alg1[i] < list_alg2[i]:
           list_rank_alg1.append(2)
           list_rank_alg2.append(3)
           list_rank_alg3.append(1)
           
        elif list_alg3[i] < list_alg1[i] and list_alg1[i] == list_alg2[i]:
           list_rank_alg1.append(2.5)
           list_rank_alg2.append(2.5)
           list_rank_alg3.append(1)
           
        # ----------

        elif list_alg3[i] < list_alg2[i] and list_alg2[i] < list_alg1[i]:
           list_rank_alg1.append(3)
           list_rank_alg2.append(2)
           list_rank_alg3.append(1)

    else:
      print("Error: Incorrect value on the parameter \"highest\"!")

    headers = ['Fold', 'K-NN', 'SVM', 'Naive Bayes']


    dict_data={headers[1]:[f"{list_alg1[0]} ({list_rank_alg1[0]})",
                           f"{list_alg1[1]} ({list_rank_alg1[1]})",
                           f"{list_alg1[2]} ({list_rank_alg1[2]})",
                           f"{list_alg1[3]} ({list_rank_alg1[3]})",
                           f"{list_alg1[4]} ({list_rank_alg1[4]})",
                           f"{list_alg1[5]} ({list_rank_alg1[5]})",
                           f"{list_alg1[6]} ({list_rank_alg1[6]})",
                           f"{list_alg1[7]} ({list_rank_alg1[7]})",
                           f"{list_alg1[8]} ({list_rank_alg1[8]})",
                           f"{list_alg1[9]} ({list_rank_alg1[9]})"],
               
               headers[2]:[f"{list_alg2[0]} ({list_rank_alg2[0]})",
                           f"{list_alg2[1]} ({list_rank_alg2[1]})",
                           f"{list_alg2[2]} ({list_rank_alg2[2]})",
                           f"{list_alg2[3]} ({list_rank_alg2[3]})",
                           f"{list_alg2[4]} ({list_rank_alg2[4]})",
                           f"{list_alg2[5]} ({list_rank_alg2[5]})",
                           f"{list_alg2[6]} ({list_rank_alg2[6]})",
                           f"{list_alg2[7]} ({list_rank_alg2[7]})",
                           f"{list_alg2[8]} ({list_rank_alg2[8]})",
                           f"{list_alg2[9]} ({list_rank_alg2[9]})"],
               
               headers[3]:[f"{list_alg3[0]} ({list_rank_alg3[0]})",
                           f"{list_alg3[1]} ({list_rank_alg3[1]})",
                           f"{list_alg3[2]} ({list_rank_alg3[2]})",
                           f"{list_alg3[3]} ({list_rank_alg3[3]})",
                           f"{list_alg3[4]} ({list_rank_alg3[4]})",
                           f"{list_alg3[5]} ({list_rank_alg3[5]})",
                           f"{list_alg3[6]} ({list_rank_alg3[6]})",
                           f"{list_alg3[7]} ({list_rank_alg3[7]})",
                           f"{list_alg3[8]} ({list_rank_alg3[8]})",
                           f"{list_alg3[9]} ({list_rank_alg3[9]})"]}

    data_df=pd.DataFrame(dict_data)

    data_df.index = np.arange(1, len(data_df) + 1)

    data_df[headers[0]]=data_df.index
    col_fold = data_df.pop(headers[0])
    data_df.insert(0, headers[0], col_fold)

    avg_rank_alg1 = sum(list_rank_alg1)/list_size
    avg_rank_alg2 = sum(list_rank_alg2)/list_size
    avg_rank_alg3 = sum(list_rank_alg3)/list_size

    avg_rank_df = pd.DataFrame([
        ['-', '-', '-', '-'],
        ['avg rank', avg_rank_alg1, avg_rank_alg2, avg_rank_alg3]],
        columns=headers)

    result_df = pd.concat([data_df, avg_rank_df])

    print("\n\nFriedman test:")
    print(tabulate(result_df, headers="keys", showindex=False, tablefmt="rst"))
    

    k = 3
    n = 10
    avg_rank = (k + 1) / 2 # = 2
    test1 = avg_rank

    test2 = n * (pow(avg_rank_alg1 - avg_rank, 2) +
                 pow(avg_rank_alg2 - avg_rank, 2) +
                 pow(avg_rank_alg3 - avg_rank, 2))
    
    sum_per_dataset = 0
    for i in range(n):
      sum_per_dataset += pow(list_rank_alg1[i] - avg_rank, 2)
      sum_per_dataset += pow(list_rank_alg2[i] - avg_rank, 2)
      sum_per_dataset += pow(list_rank_alg3[i] - avg_rank, 2)

    test3 = sum_per_dataset / (n * (k - 1))

    friedman_statistic = test2 / test3 # ratio

    print("\nAverage rank:", test1)
    print("Second quantity:", test2)
    print("Third quantity:", test3)
    print("Friedman statistic (ratio):", friedman_statistic)

    critical_value = 6.20
    print("Critical value:", critical_value)

    if friedman_statistic > critical_value:
      print("Friedman statistic exceed the critical value.\nNull hypothesis is rejected. Performing Nemenyi test.")
      dict_avg_ranks = {headers[1]:avg_rank_alg1 , headers[2]:avg_rank_alg2, headers[3]:avg_rank_alg3}
      Nemenyi_test(k, n, dict_avg_ranks)

    else:
      print("Friedman statistic does not exceed the critical value.\nNull hypothesis is accepted.")

  else:
    print("Error: Input lists not the same length!")

---
Results for computational performance in terms of training time:

In [50]:
print("Results for computational performance in terms of training time:")
cross_validation(list_kfold_time1, list_kfold_time2, list_kfold_time3)
Friedman_test(list_kfold_time1, list_kfold_time2, list_kfold_time3, False)

Results for computational performance in terms of training time:
======  =====================  ===================  =====================
Fold    K-NN                   SVM                  Naive Bayes
======  =====================  ===================  =====================
1       0.002456               0.616917             0.004738
2       0.001968               0.557984             0.005442
3       0.001886               0.538859             0.00509
4       0.001923               0.546                0.004572
5       0.003205               0.536923             0.007256
6       0.001962               0.486254             0.004536
7       0.00185                0.5091               0.004672
8       0.001926               0.547481             0.004869
9       0.00191                0.563842             0.005107
10      0.002028               0.57983              0.004619
-       -                      -                    -
avg     0.0021114              0.548319             0.005090

---
Results for for predictive performance based on accuracy:

In [51]:
print("Results for predictive performance based on accuracy:")
cross_validation(list_kfold_acc1, list_kfold_acc3, list_kfold_acc3)
Friedman_test(list_kfold_acc1, list_kfold_acc2, list_kfold_acc3, True)

Results for predictive performance based on accuracy:
======  ====================  ====================  ====================
Fold    K-NN                  SVM                   Naive Bayes
======  ====================  ====================  ====================
1       0.9108695652173913    0.8108695652173913    0.8108695652173913
2       0.8978260869565218    0.8304347826086956    0.8304347826086956
3       0.9282608695652174    0.8021739130434783    0.8021739130434783
4       0.9217391304347826    0.8173913043478261    0.8173913043478261
5       0.9347826086956522    0.8108695652173913    0.8108695652173913
6       0.9195652173913044    0.8173913043478261    0.8173913043478261
7       0.908695652173913     0.7913043478260869    0.7913043478260869
8       0.9217391304347826    0.8130434782608695    0.8130434782608695
9       0.9239130434782609    0.7934782608695652    0.7934782608695652
10      0.9065217391304348    0.8478260869565217    0.8478260869565217
-       -                 

---
Results for predictive performance based on F-measure:

In [52]:
print("Results of predictive performance based on F-measure:")
cross_validation(list_kfold_fm1,list_kfold_fm2, list_kfold_fm3)
Friedman_test(list_kfold_fm1, list_kfold_fm2, list_kfold_fm3, True)

Results of predictive performance based on F-measure:
======  ===================  ====================  ==================
Fold    K-NN                 SVM                   Naive Bayes
======  ===================  ====================  ==================
1       0.9109949616084221   0.8907172034181373    0.8125889246526776
2       0.897481897194985    0.8780440988886962    0.8336733643228638
3       0.9285237550737282   0.9058779999172353    0.8042133572389063
4       0.9216425791268141   0.8907580325224241    0.8170495021344142
5       0.9347221400248564   0.8856844305120168    0.8124436266722309
6       0.9193591190434008   0.8948302072739854    0.8191593239924887
7       0.9088610586011342   0.8945740398415724    0.7931891977508515
8       0.9213763184100084   0.8792706054543217    0.8140947304041611
9       0.9243982356647762   0.8761659927785819    0.7969923136605246
10      0.9060395237405362   0.8771499476608144    0.8489579208574219
-       -                    -             